# Results
> code for plotting results and comparing with MDS and ERA gap filling

In [ ]:
#| hide
#| default_exp kalman.results

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import altair as alt

In [ ]:
import altair

In [ ]:
#| export
from fastcore.test import *
from fastcore.basics import *
from meteo_imp.utils import *
from meteo_imp.gaussian import *
from meteo_imp.data_preparation import MeteoDataTest
from meteo_imp.kalman.filter import *
from meteo_imp.kalman.filter import get_test_data
from meteo_imp.data import *
from meteo_imp.kalman.fastai import *
from meteo_imp.kalman.fastai import _n_tuple
from fastcore.transform import *
from fastai.learner import *
from pyprojroot import here

import pykalman
from typing import *

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import torch
from torch import Tensor
from torch.distributions import MultivariateNormal

from timeit import timeit
import polars as pl
import altair as alt

from tqdm.auto import tqdm

import io
from contextlib import redirect_stderr
import random

from dataclasses import dataclass
from functools import partial
from itertools import zip_longest

load data for testing

In [ ]:
reset_seed()

In [ ]:
hai = pd.read_parquet(hai_big_path)
hai_era = pd.read_parquet(hai_era_big_path)

In [ ]:
dls = imp_dataloader(hai, hai_era, var_sel = 'TA', gap_len=5, block_len=100, control_lags = [1], n_rep=1, bs=1).cpu()

In [ ]:
item = MeteoImpItem(1,2, 'TA', gap_len=10)
items = [item]

In [ ]:
targ = orig_target(dls, items)[0]

In [ ]:
input, _ = one_batch_with_items(dls, items)

## Imputation Methods

### Kalman Filter

take a `MeteoImp Df` and gap fill it using the given model

In [ ]:
model = torch.load(here("analysis/results/trained_13feb/1_gap_varying_12-336_v1.pickle"))

In [ ]:
preds, targs = predict_items(model, dls=dls, items = items)
preds[0].mean[targs[0].mask] = targs[0].data[targs[0].mask]

#### KalmanImputation

In [ ]:
_n_tuple

<function meteo_imp.kalman.fastai._n_tuple(x, n, unsqueeze=True)>

In [ ]:
input[1].shape

torch.Size([1, 100, 9])

In [ ]:
preds_raw = model(input)

In [ ]:
len(preds_raw.mean[0])

10

In [ ]:
#| export
def _extract_var(preds, var_idx, max_len):
    "extract prediction only from one var"
    preds_new = []
    for b_pred in preds:
        b_pred_new = []
        for pred in b_pred:
            if len(pred) == 1: pred_new = pred
            elif len(pred) == max_len: pred_new = pred[var_idx:var_idx+1] if pred.dim() == 1 else pred[var_idx:var_idx+1, var_idx:var_idx+1]
            else: raise ValueError("supports only gaps for 1 or all variables")
            b_pred_new.append(pred_new)
        preds_new.append(b_pred_new)
    return preds_new
         

In [ ]:
_extract_var(preds_raw.mean, 1, 7)

[[tensor([-0.7555], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([-0.7765], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([-0.8153], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([-0.8653], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([-0.8660], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([-0.8712], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([-0.8551], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([-0.8420], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([-0.8442], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([-0.8536], dtype=torch.float64, grad_fn=<IndexBackward0>)]]

In [ ]:
_extract_var(preds_raw.cov, 1, 7)

[[tensor([[0.0227]], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([[0.0307]], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([[0.0359]], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([[0.0392]], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([[0.0409]], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([[0.0409]], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([[0.0393]], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([[0.0361]], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([[0.0310]], dtype=torch.float64, grad_fn=<IndexBackward0>),
  tensor([[0.0230]], dtype=torch.float64, grad_fn=<IndexBackward0>)]]

In [ ]:
items_all = [MeteoImpItem(1,2, list(hai.columns), gap_len=3)] * 3

In [ ]:
input_all, _ = one_batch_with_items(dls, items_all)

In [ ]:
preds_raw_all = model(input_all)

In [ ]:
_extract_var(preds_raw_all.mean, 3, 9)

[[tensor([-0.5650], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([-0.5887], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([-0.6259], dtype=torch.float64, grad_fn=<SliceBackward0>)],
 [tensor([-0.5650], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([-0.5887], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([-0.6259], dtype=torch.float64, grad_fn=<SliceBackward0>)],
 [tensor([-0.5650], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([-0.5887], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([-0.6259], dtype=torch.float64, grad_fn=<SliceBackward0>)]]

In [ ]:
buffer_pred(preds_raw_all.mean, input_all[1]).shape

torch.Size([3, 100, 9])

In [ ]:
_extract_var(preds_raw_all.cov, 3, 9)

[[tensor([[0.1189]], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([[0.1717]], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([[0.1251]], dtype=torch.float64, grad_fn=<SliceBackward0>)],
 [tensor([[0.1189]], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([[0.1717]], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([[0.1251]], dtype=torch.float64, grad_fn=<SliceBackward0>)],
 [tensor([[0.1189]], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([[0.1717]], dtype=torch.float64, grad_fn=<SliceBackward0>),
  tensor([[0.1251]], dtype=torch.float64, grad_fn=<SliceBackward0>)]]

In [ ]:
#| export
class PredictLossVar:
    """loss (negative log likelihood) for only for one variable for each batch"""
    def __init__(self, only_gap:bool, var: int):
        self.loss_func = KalmanLoss(only_gap)
        self.var = var
    def __call__(self, preds, targs):
        sel_idx = [idx for idx in range(targs[1].shape[-1]) if idx != self.var]
        mask_new = targs[1].clone()
        mask_new[:, :, sel_idx] = True # make all other variables present
        targs_new = (targs[0], mask_new, targs[2])
        preds_new = (_extract_var(preds.mean, self.var, targs[1].shape[-1]), _extract_var(preds.cov, self.var, targs[1].shape[-1])) 
        # return preds_new
        # return self.loss_func(preds_new, targs_new)
        losses = []
        for i in range(len(preds_new[0])):
            losses.append(self.loss_func(_n_tuple(preds_new, i), _n_tuple(targs_new, i)))
        return losses

In [ ]:
PredictLossVar(True, 3)(preds_raw_all, input_all)

[tensor(-0.2252, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(-0.2252, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(-0.2252, dtype=torch.float64, grad_fn=<MeanBackward0>)]

In [ ]:
input_all[1].shape

torch.Size([3, 100, 9])

In [ ]:
#| export
class PredictLikelihoodVar:
    """mean between timesteps of Likelihood for only for one variable for each batch"""
    def __init__(self, only_gap:bool, var: int):
        self.loss_func = KalmanLoss(only_gap, reduction_inbatch='none', reduction = 'none')
        self.var = var
    def __call__(self, preds, targs):
        sel_idx = [idx for idx in range(targs[1].shape[-1]) if idx != self.var]
        mask_new = targs[1].clone()
        mask_new[:, :, sel_idx] = True # make all other variables present
        targs_new = (targs[0], mask_new, targs[2])
        preds_new = (_extract_var(preds.mean, self.var, targs[1].shape[-1]), _extract_var(preds.cov, self.var, targs[1].shape[-1])) 
        # return preds_new
        # return self.loss_func(preds_new, targs_new)
        likelihoods = []
        for i in range(len(preds_new[0])):
            loss = self.loss_func(_n_tuple(preds_new, i), _n_tuple(targs_new, i))[0]
            lh = (torch.exp(-loss[0])).mean()
            likelihoods.append(lh)
        return likelihoods

In [ ]:
PredictLikelihoodVar(True, 3)(preds_raw_all, input_all)

[tensor(1.1569, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(1.1569, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(1.1569, dtype=torch.float64, grad_fn=<MeanBackward0>)]

In [ ]:
#| export
class MultiMetrics():
    def __init__(self, **metrics): self.metrics = metrics
    def __call__(self, preds, targs): return {name: metric(preds, targs) for name, metric in self.metrics.items()}

In [ ]:
#| export
class KalmanImputation:
    name = "KalmanFilter"
    def __init__(self, model): store_attr()
    def __call__(self, item, dls):
        pred, targ = predict_items(self.model, dls=dls, items = [item])
        pred[0].mean[targ[0].mask] = targ[0].data[targ[0].mask]
        return pred[0].mean
    def preds_all(self, items, dls):
        return predict_items(self.model, dls=dls, items = items)
    def preds_all_loss(self, items, dls, var):
        return predict_items(self.model, dls=dls, items = items, metric_fn = PredictLossVar(only_gap=self.model.pred_only_gap, var = var))
    def preds_all_metrics(self, items, dls, metrics):
        return predict_items(self.model, dls=dls, items = items, metric_fn = metrics)

In [ ]:
k_imp = KalmanImputation(model)

In [ ]:
pred = k_imp(item, dls=dls)

In [ ]:
display_as_row({'pred': pred[45:55],'data': targ.data[45:55], 'mask': targ.mask[45:55]}, hide_idx=False)

,TA,SW_IN,LW_IN,VPD,WS,PA,P,SWC,TS
time,,,,,,,,,
2000-01-04 02:30:00,2.3467,0.0000,304.7310,0.9520,5.3900,96.3000,0.0000,43.0800,1.3100
2000-01-04 03:00:00,2.1803,0.0000,304.7310,0.9710,5.5900,96.3000,0.0000,43.0800,1.3100
2000-01-04 03:30:00,1.8728,0.0000,304.7310,0.9580,5.6700,96.2900,0.0000,43.0800,1.3100
2000-01-04 04:00:00,1.4769,0.0000,304.7310,0.9380,6.6100,96.2600,0.0000,43.0800,1.3100
2000-01-04 04:30:00,1.4710,0.0000,315.0050,0.7850,6.6500,96.2600,0.0000,43.0800,1.3100
2000-01-04 05:00:00,1.4303,0.0000,315.0050,0.5290,6.1200,96.2700,0.0000,43.0800,1.3000
2000-01-04 05:30:00,1.5573,0.0000,315.0050,0.4500,5.5800,96.2600,0.2900,43.1300,1.3000
2000-01-04 06:00:00,1.6616,0.0000,315.0050,0.4990,4.7000,96.2500,0.0000,43.1300,1.2900
2000-01-04 06:30:00,1.6438,0.0000,315.0050,0.3820,5.0700,96.2300,0.0000,43.1300,1.2900


#### Kalman Imputation - Specialized Variables

In [ ]:
#| export
class KalmanImputationVar:
    name = "KalmanFilter"
    def __init__(self, models # dataframe with 2 columns `model` and `var`
                ): store_attr()
    def __call__(self, var, item, dls):
        model = self._select_model(var)
        pred, targ = predict_items(model, dls=dls, items = [item])
        pred[0].mean[targ[0].mask] = targ[0].data[targ[0].mask]
        return pred[0].mean
    def _select_model(self, var):
        return self.models[self.models['var'] == var].model.iloc[0]
    def preds_all(self, var:str, items:list, dls):
        model = self._select_model(var)
        return predict_items(model, dls=dls, items = items)

In [ ]:
k_impVar = KalmanImputationVar(pd.DataFrame({'model': [model], 'var': 'TA'}))

In [ ]:
pred = k_impVar(var= 'TA', item=item, dls=dls)

In [ ]:
display_as_row({'pred': pred[45:55],'data': targ.data[45:55], 'mask': targ.mask[45:55]}, hide_idx=False)

,TA,SW_IN,LW_IN,VPD,WS,PA,P,SWC,TS
time,,,,,,,,,
2000-01-04 02:30:00,2.3467,0.0000,304.7310,0.9520,5.3900,96.3000,0.0000,43.0800,1.3100
2000-01-04 03:00:00,2.1803,0.0000,304.7310,0.9710,5.5900,96.3000,0.0000,43.0800,1.3100
2000-01-04 03:30:00,1.8728,0.0000,304.7310,0.9580,5.6700,96.2900,0.0000,43.0800,1.3100
2000-01-04 04:00:00,1.4769,0.0000,304.7310,0.9380,6.6100,96.2600,0.0000,43.0800,1.3100
2000-01-04 04:30:00,1.4710,0.0000,315.0050,0.7850,6.6500,96.2600,0.0000,43.0800,1.3100
2000-01-04 05:00:00,1.4303,0.0000,315.0050,0.5290,6.1200,96.2700,0.0000,43.0800,1.3000
2000-01-04 05:30:00,1.5573,0.0000,315.0050,0.4500,5.5800,96.2600,0.2900,43.1300,1.3000
2000-01-04 06:00:00,1.6616,0.0000,315.0050,0.4990,4.7000,96.2500,0.0000,43.1300,1.2900
2000-01-04 06:30:00,1.6438,0.0000,315.0050,0.3820,5.0700,96.2300,0.0000,43.1300,1.2900


### MDS

Need to call R from python

In [ ]:
#| export
import rpy2.robjects
from rpy2.robjects import r
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [ ]:
r.print('here::here("R/REddyProc_tools.R")')

[1] "here::here(\"R/REddyProc_tools.R\")"


"'here::here(""R/REddyProc_tools.R"")'"


In [ ]:
#| export
path_r = str(here("R/REddyProc_tools.R"))

r.source(path_r);

In [ ]:
#| export
def importr_install(pkg):
    try:
        importr(pkg)
    except:
        utils = importr('utils')
        utils.chooseCRANmirror(ind=1)
        utils.install_packages(pkg)
        importr(pkg) 

In [ ]:
#| export
def setupR():
    importr_install('tidyverse')
    importr_install('REddyProc')
    importr('lubridate')
    path_r = str(here("R/REddyProc_tools.R"))
    r.source(path_r) # R functions
    r("""toR_timestamp <- function(x){
   x$TIMESTAMP_END = as_datetime(x$TIMESTAMP_END) 
    x
     }""")

In [ ]:
#| export
def pd2R(x):
    with ro.default_converter + pandas2ri.converter:
        return ro.conversion.py2rpy(x) 

def R2pd(x):
    with ro.default_converter + pandas2ri.converter:
        return ro.conversion.rpy2py(x) 

In [ ]:
#| export
setupR() #start the R process and load depenencies

#### Experiment

The time series needs to be at least 90 days long, we we add 45 days before and after the gap

There is a problem with conversions of timestamps between R and Python, so convert to string in Python and then back to datetime in R

In [ ]:
#| export
def add_buffer(index, inner_index, n):
    """Adds  a buffer of after and after index so length is at least """
    start = int(np.argwhere(index == inner_index[0]).squeeze())
    end = int(np.argwhere(index == inner_index[-1]).squeeze())
    start = start - n //2
    end = end + n//2
    if start < 0:
        end += -start
        start = 0
    if end > len(index):
        start -= end - len(index)
        end = len(index)
    
    index = index[start:end]
    
    assert len(index) > n
    
    return index

In [ ]:
add_buffer(hai.index, hai.index[:100], 50)
add_buffer(hai.index, hai.index[-50:], 50);

In [ ]:
#| export
def item2REddy(item, var, df):
    " Add context around item for supporting REddyProc"
    index = add_buffer(df.index, item.data.index, 90 * 24 * 2)
    REddy_df = df.loc[index].assign(gap = (~item.mask[var]).astype(int)).fillna({'gap': 0})
    return REddy_df.reset_index().astype({'time': str}).rename(columns={'time': 'TIMESTAMP_END'})

In [ ]:
REddy_df = item2REddy(targ, 'TA', hai)

REddy_df_r = r.toR_timestamp(pd2R(REddy_df))

filled = R2pd(r.fill_gaps_EProc(REddy_df_r, "TA"))

R[write to console]: New sEddyProc class for site 'ID'

R[write to console]: Initialized variable 'TA' with 10 real gaps for gap filling.

R[write to console]: Limited MDS algorithm for gap filling of 'TA.gap_0' with LUT(SW_IN only) and MDC.

R[write to console]: Look up table with window size of 7 days with SW_IN

R[write to console]: 10

R[write to console]: Finished gap filling of 'TA' in 0 seconds. Artificial gaps filled: 4419, real gaps filled: 10, unfilled (long) gaps: 0.



In [ ]:
filled

,TA_orig,TA_f,TA_fqc,TA_fall,TA_fall_qc,TA_fnum,TA_fsd,TA_fmeth,TA_fwin
1,-0.60,-0.60,0.0,-0.60,NaN,NaN,NaN,NaN,NaN
2,-0.65,-0.65,0.0,-0.65,NaN,NaN,NaN,NaN,NaN
3,-0.58,-0.58,0.0,-0.58,NaN,NaN,NaN,NaN,NaN
4,-0.51,-0.51,0.0,-0.51,NaN,NaN,NaN,NaN,NaN
5,-0.49,-0.49,0.0,-0.49,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4415,3.30,3.30,0.0,3.30,NaN,NaN,NaN,NaN,NaN
4416,3.10,3.10,0.0,3.10,NaN,NaN,NaN,NaN,NaN
4417,3.05,3.05,0.0,3.05,NaN,NaN,NaN,NaN,NaN
4418,3.05,3.05,0.0,3.05,NaN,NaN,NaN,NaN,NaN


In [ ]:
REddy_df.set_index("TIMESTAMP_END") 

,TA,SW_IN,LW_IN,VPD,WS,PA,P,SWC,TS,gap
TIMESTAMP_END,,,,,,,,,,
2000-01-01 00:30:00,-0.60,0.0,302.475,0.222,2.05,96.63,0.0,43.00,1.44,0.0
2000-01-01 01:00:00,-0.65,0.0,302.475,0.122,2.53,96.58,0.0,43.00,1.43,0.0
2000-01-01 01:30:00,-0.58,0.0,301.677,0.090,3.15,96.56,0.0,43.00,1.43,0.0
2000-01-01 02:00:00,-0.51,0.0,301.677,0.110,3.12,96.56,0.0,43.00,1.45,0.0
2000-01-01 02:30:00,-0.49,0.0,301.677,0.102,3.04,96.57,0.0,43.00,1.44,0.0
...,...,...,...,...,...,...,...,...,...,...
2000-04-01 23:30:00,3.30,0.0,283.856,1.092,2.50,95.06,0.0,44.96,4.32,0.0
2000-04-02 00:00:00,3.10,0.0,283.856,1.013,2.09,95.04,0.0,44.96,4.22,0.0
2000-04-02 00:30:00,3.05,0.0,283.856,0.964,2.42,95.04,0.0,44.96,4.13,0.0


In [ ]:
filled = filled.set_index(pd.to_datetime(REddy_df.TIMESTAMP_END))

filled_item = filled.loc[targ.data.index]

pred = targ.data.copy()
pred.loc[~targ.mask["TA"], "TA"] = filled_item["TA_f"][~targ.mask["TA"]]

In [ ]:
pred[45:55], targ.data[45:55]

(                           TA  SW_IN    LW_IN    VPD    WS     PA     P  \
 time                                                                      
 2000-01-04 02:30:00  1.853612    0.0  304.731  0.952  5.39  96.30  0.00   
 2000-01-04 03:00:00  1.837083    0.0  304.731  0.971  5.59  96.30  0.00   
 2000-01-04 03:30:00  1.823985    0.0  304.731  0.958  5.67  96.29  0.00   
 2000-01-04 04:00:00  1.809805    0.0  304.731  0.938  6.61  96.26  0.00   
 2000-01-04 04:30:00  1.794282    0.0  315.005  0.785  6.65  96.26  0.00   
 2000-01-04 05:00:00  1.783447    0.0  315.005  0.529  6.12  96.27  0.00   
 2000-01-04 05:30:00  1.771525    0.0  315.005  0.450  5.58  96.26  0.29   
 2000-01-04 06:00:00  1.760362    0.0  315.005  0.499  4.70  96.25  0.00   
 2000-01-04 06:30:00  1.748410    0.0  315.005  0.382  5.07  96.23  0.00   
 2000-01-04 07:00:00  1.737933    0.0  315.005  0.313  5.23  96.24  0.00   
 
                        SWC    TS  
 time                              
 2000-01-04 02

In [ ]:
#| export
def gap_fill_item(item, REddy_df, var, filled):
    
    filled = filled.set_index(pd.to_datetime(REddy_df.TIMESTAMP_END))
    filled_item = filled.loc[item.data.index]

    pred = item.data.copy()
    pred.loc[~item.mask[var], var] = filled_item[f"{var}_f"][~item.mask[var]]
    return pred

In [ ]:
gap_fill_item(targ, REddy_df, "TA", filled)


,TA,SW_IN,LW_IN,VPD,WS,PA,P,SWC,TS
time,,,,,,,,,
2000-01-03 04:00:00,0.81,0.0,304.148,0.000,3.53,97.00,0.0,43.09,1.37
2000-01-03 04:30:00,0.95,0.0,304.382,0.000,3.99,96.98,0.0,43.09,1.37
2000-01-03 05:00:00,1.09,0.0,304.382,0.000,3.61,96.96,0.0,43.09,1.37
2000-01-03 05:30:00,1.18,0.0,304.382,0.009,3.90,96.93,0.0,43.09,1.37
2000-01-03 06:00:00,1.35,0.0,304.382,0.061,4.17,96.91,0.0,43.09,1.38
...,...,...,...,...,...,...,...,...,...
2000-01-05 03:30:00,4.62,0.0,330.202,1.162,6.53,95.91,0.0,44.12,1.82
2000-01-05 04:00:00,4.51,0.0,330.202,1.636,5.76,95.94,0.0,44.12,1.85
2000-01-05 04:30:00,4.11,0.0,299.320,1.746,5.79,95.98,0.0,44.12,1.85


#### MDSImputation

In [ ]:
#| export
class MDSImputation:
    name = "MDS"
    def __init__(self, var, df):
        store_attr()
        self.out = io.StringIO()
    def __call__(self, item):
        REddy_df = item2REddy(item, self.var, self.df)
        REddy_df_r = r.toR_timestamp(pd2R(REddy_df))
        with redirect_stderr(self.out):
            filled = R2pd(r.fill_gaps_EProc(REddy_df_r, self.var))
        return gap_fill_item(item, REddy_df, self.var, filled)

In [ ]:
mds_imp = MDSImputation('TA', hai)

In [ ]:
mds_imp(targ)

,TA,SW_IN,LW_IN,VPD,WS,PA,P,SWC,TS
time,,,,,,,,,
2000-01-03 04:00:00,0.81,0.0,304.148,0.000,3.53,97.00,0.0,43.09,1.37
2000-01-03 04:30:00,0.95,0.0,304.382,0.000,3.99,96.98,0.0,43.09,1.37
2000-01-03 05:00:00,1.09,0.0,304.382,0.000,3.61,96.96,0.0,43.09,1.37
2000-01-03 05:30:00,1.18,0.0,304.382,0.009,3.90,96.93,0.0,43.09,1.37
2000-01-03 06:00:00,1.35,0.0,304.382,0.061,4.17,96.91,0.0,43.09,1.38
...,...,...,...,...,...,...,...,...,...
2000-01-05 03:30:00,4.62,0.0,330.202,1.162,6.53,95.91,0.0,44.12,1.82
2000-01-05 04:00:00,4.51,0.0,330.202,1.636,5.76,95.94,0.0,44.12,1.85
2000-01-05 04:30:00,4.11,0.0,299.320,1.746,5.79,95.98,0.0,44.12,1.85


### ERA Imputation

In [ ]:
#| export
class ERAImputation:
    name = "ERA"
    def __call__(self, item):
        pred = item.control.copy()
        names = [col for col in pred.columns if not col.endswith("_lag_1")]
        pred = pred.filter(names)
        pred = pred.rename(columns=lambda x: x.replace("_ERA", ""))
        # columns that cannot be predicted get a NA        
        for col in item.data.columns:
            if col not in pred.columns:
                pred[col] = np.nan 
        return pred

In [ ]:
targ.data.columns

Index(['TA', 'SW_IN', 'LW_IN', 'VPD', 'WS', 'PA', 'P', 'SWC', 'TS'], dtype='object')

In [ ]:
era_imp = ERAImputation()

In [ ]:
era_imp(targ)

,TA,SW_IN,VPD,PA,P,WS,LW_IN,SWC,TS
time,,,,,,,,,
2000-01-03 04:00:00,1.702,0.0,0.693,97.016,0.000,2.948,304.148,NaN,NaN
2000-01-03 04:30:00,1.762,0.0,0.691,97.002,0.026,2.989,304.382,NaN,NaN
2000-01-03 05:00:00,1.736,0.0,0.697,96.988,0.000,3.015,304.382,NaN,NaN
2000-01-03 05:30:00,1.711,0.0,0.703,96.974,0.000,3.041,304.382,NaN,NaN
2000-01-03 06:00:00,1.686,0.0,0.709,96.960,0.000,3.066,304.382,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2000-01-05 03:30:00,4.945,0.0,0.774,95.910,0.252,6.107,330.202,NaN,NaN
2000-01-05 04:00:00,4.789,0.0,0.758,95.926,0.252,6.189,330.202,NaN,NaN
2000-01-05 04:30:00,4.632,0.0,0.741,95.942,0.044,6.270,299.320,NaN,NaN


$\sigma$

## Metrics

In [ ]:
#| export
class MaskedMetric:
    def __init__(self, metric): store_attr()
    def __call__(self, targ, pred):
        if isinstance(pred, NormalsDf): pred = pred.mean
        row_sel, col_sel = ~targ.mask.all(1), ~targ.mask.all(0)
        assert not targ.mask.loc[row_sel,col_sel].any().all() # gap is a rectangle
        data, pred = targ.data.loc[row_sel,col_sel], pred.loc[row_sel,col_sel]
        return self.metric(data, pred) if not np.isnan(pred).all().all() else np.array([np.nan])

In [ ]:
#| export
from sklearn.metrics import mean_squared_error

In [ ]:
#| export
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred, multioutput='raw_values'))

In [ ]:
#| export
def normalize(x, mean, std): return (x - mean) / std 
class NormalizedMetric:
    def __init__(self, metric: MaskedMetric, mean, std):
        self.mean = np.array(mean)
        self.std = np.array(std)
        self.metric = metric
    def __call__(self, targ, pred, var:int|None = None):
        targ = targ.copy()
        norm_conf = (self.mean if var is None else self.mean[var], self.std if var is None else self.std[var])
        targ.data = normalize(targ.data, *norm_conf)
        if isinstance(pred, NormalsDf): pred = pred.mean
        pred = normalize(pred, *norm_conf)
        return self.metric(targ, pred)

In [ ]:
#| export
rmse_mask = MaskedMetric(rmse)

In [ ]:
rmse_mask(targ, pred)

array([0.3698457])

### Validation Imputations

checks that the values of the imputation makes sense

this is the average error for ERA

In [ ]:
np.sqrt((hai['TA'] - hai_era.loc[hai.index]['TA_ERA']).pow(2).mean())

1.8708585984196093

In [ ]:
rmse_mask(targ, ERAImputation()(targ))

array([0.29919709])

In [ ]:
err_era = []
for _ in range(100):
    items = random.choices(dls.items, k = 1)
    targ = orig_target(dls, items)[0]
    err_era.append(rmse_mask(targ, ERAImputation()(targ)))

In [ ]:
np.array(err_era).mean()

1.3839633283333717

## Comparison

Take a variable to be filled, makes an artificial gap of given len, tries to fill with 3 methods and return metrics for each of them, repeat `n_rep` times

In [ ]:
#| export
import random
import polars as pl
from tqdm.auto import tqdm

### prep visualization

In [ ]:
#| export
var_type = CategoricalDtype(categories=['TA', 'SW_IN', 'LW_IN', 'VPD', 'WS', 'PA', 'P', 'SWC', 'TS'], ordered=True)

method_type = CategoricalDtype(categories=["KalmanFilter", "ERA", "MDS"], ordered=True)

In [ ]:
#| export
def _as_category(df: pd.DataFrame):
    df = df.assign(
        var = df['var'].astype(var_type),
        gap_len = df['gap_len'].astype("category"))
    if 'method' in df.columns: df = df.assign(method = df['method'].astype(method_type))
    return df.sort_values(['var', 'gap_len', 'method'] if 'method' in df.columns else ['var', 'gap_len'] )

In [ ]:
#| export
def gaplen2hours(x):
    return x.assign(gap_len = (x.gap_len / 2))#.map(lambda x: f"{int(x) if round(x) == x else x} h")) 

In [ ]:
#| export
def prep_df(df): return _as_category(gaplen2hours(df)) # what a bad name

### Aggregation

In [ ]:
hai.std(axis=0)['TA']

7.924610764367695

In [ ]:
get_stats(hai)

(tensor([8.3339e+00, 1.2096e+02, 3.1150e+02, 3.3807e+00, 3.1800e+00, 9.5962e+01,
         4.3427e-02, 3.4843e+01, 7.9349e+00], dtype=torch.float64),
 tensor([  7.9246, 204.0026,  41.9557,   4.3684,   1.6254,   0.8552,   0.2803,
           8.9131,   5.6586], dtype=torch.float64))

$$ \text{RMSE}_\text{stand} = \frac{1}{\sigma}\text{RMSE}$$

In [ ]:
#| export
def timeseriesAgg(targ, pred, *args): return {'pred': [pred], 'targ': [targ]}

class RMSEAgg:
    """Aggregate to rmse and normalized rmse"""
    def __init__(self, df):
        self.std = df.std(axis=0)
    def __call__(self, targ, pred, var):
        rmse = rmse_mask(targ, pred).item()
        rmse_stand = rmse / self.std[var]
        return {'rmse': rmse,'rmse_stand': rmse_stand,}

### Imp Methods

In [ ]:
#| export
class ImpComparison():
    def __init__(self, models: pd.DataFrame, df, control, block_len, rmse=True, time_series = False):
        store_attr()
        self.k_imp = KalmanImputationVar(models)
        self.era_imp = ERAImputation()
        self.mds_imp = MDSImputation("", df)
        self.methods = [self.k_imp, self.era_imp, self.mds_imp]
        aggrs = []
        if rmse: aggrs.append(RMSEAgg(df))
        if time_series: aggrs.append(timeseriesAgg) 
        self.aggrs = aggrs
        
    def _compare_single(self, gap_len, var, n_rep):
        """Compares `n_rep` times the imputation methods, for gap in `var` with len `gap_len`"""
        dls = imp_dataloader(self.df, self.control, var_sel = var, gap_len=gap_len, block_len=self.block_len, control_lags = [1], n_rep=1, bs=1).cpu()
        self.mds_imp.var = var
        
        outs = []
        for i in tqdm(range(n_rep), leave=False):
            item = random.choice(dls.items)
            pred_k, targ = self.k_imp.preds_all(var = var, items = [item], dls=dls)
            pred_k, targ = pred_k[0], targ[0]
            for imp in self.methods:
                pred = imp(targ) if imp is not self.k_imp else pred_k
                out = {
                    'method': imp.name,
                    'var': var,
                    'gap_len': gap_len,
                    'idx_rep': i,
                }
                for aggr in self.aggrs:
                    out = out | aggr(targ, pred, var)
                outs.append(out)
        return pd.DataFrame(outs)
    
    def compare(self, gap_len, var, n_rep, raw=False):
        """Compare imputation performance for all combination of parameters"""
        arg_sets = list(product_dict(gap_len=tuplify(gap_len), var=tuplify(var)))
        out = []
        for arg_set in tqdm(arg_sets):
            out.append(self._compare_single(**arg_set, n_rep=n_rep))
        return prep_df(pd.concat(out)) if not raw else pd.concat(out)
        

In [ ]:
base_path = here("analysis/results/trained_14feb")

In [ ]:
#| export
def l_model(x): return torch.load(base_path / x)

In [ ]:
models_var = [
    {'var': 'TA',    'model': l_model("TA_specialized_gap_12-336_v2.pickle.pickle")},
    {'var': 'SW_IN', 'model': l_model("SW_IN_specialized_gap_12-336_v2.pickle.pickle")},
    {'var': 'LW_IN', 'model': l_model("LW_IN_specialized_gap_12-336_v1.pickle")},
    {'var': 'VPD',   'model': l_model("VPD_specialized_gap_12-336_v1.pickle")},
    {'var': 'WS',    'model': l_model("VPD_specialized_gap_12-336_v1.pickle")},
    {'var': 'PA',    'model': l_model("VPD_specialized_gap_12-336_v3.pickle.pickle")},
    {'var': 'P',     'model': l_model("P_specialized_gap_12-336_v1.pickle")},
    {'var': 'TS',    'model': l_model("TS_specialized_gap_12-336_v2.pickle.pickle")},
    {'var': 'SWC',   'model': l_model("SWC_specialized_gap_12-336_v2.pickle.pickle")},
]
models_var = pd.DataFrame.from_records(models_var)

In [ ]:
comp = ImpComparison(models = models_var, df = hai, control = hai_era, block_len = 100)

In [ ]:
data_results = comp.compare(gap_len = [5,10,20, 45], var=["TA", "SW_IN"], n_rep=5) 

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
data_results.head()

,method,var,gap_len,idx_rep,rmse,rmse_stand
0,KalmanFilter,TA,2.5,0,0.458599,0.057870
3,KalmanFilter,TA,2.5,1,0.156532,0.019753
6,KalmanFilter,TA,2.5,2,0.174399,0.022007
9,KalmanFilter,TA,2.5,3,0.361323,0.045595
12,KalmanFilter,TA,2.5,4,0.655402,0.082705


In [ ]:
data_results.columns

Index(['method', 'var', 'gap_len', 'idx_rep', 'rmse', 'rmse_stand'], dtype='object')

### Kalman Filters

In [ ]:
#| export
class KalmanImpComparison():
    """Compare different Kalman filters"""
    def __init__(self,
                 models: pd.DataFrame, # model, gap_single_var, 
                 df: pd.DataFrame,
                 control: pd.DataFrame,
                 block_len:int,
                 rmse:bool=True,
                 time_series:bool = False):
        store_attr()
        self.imps = models.assign(imp = models.model.map(lambda model: KalmanImputation(model)))
        self.columns = list(df.columns)
        aggrs = []
        if rmse: aggrs.append(RMSEAgg(df))
        if time_series: aggrs.append(timeseriesAgg) 
        self.aggrs = aggrs
        
    def _compare_single(self, n_rep:int, gap_len:int, var:list[str]):
        """Compares `n_rep` times the imputation methods, for gap in `var` with len `gap_len`"""
        outs = []
        imps = imps = self.imps[self.imps['var'] == var] if 'var' in self.imps.columns else self.imps
        
       
        dls = imp_dataloader(self.df, self.control,
                                 var_sel = var,
                                 gap_len=gap_len, block_len=self.block_len, control_lags = [1], n_rep=1, bs=1,
                                 shifts = gen_shifts(50),
                                ).cpu()

        items_orig = random.choices(dls.items, k=n_rep)
             
        for (_,imp) in imps.iterrows():
            
            if not ( imp.gap_single_var if hasattr(imp, 'gap_single_var') else True):
                items = [MeteoImpItem(i = i.i, shift = i.shift, var_sel = self.columns, gap_len = i.gap_len) for i in items_orig]
            else:
                items = items_orig.copy()
            

            var_idx = _index_var(self.df, [var])[0]
            metrics_fn = MultiMetrics(loss = PredictLossVar(only_gap=True, var = var_idx), likelihood = PredictLikelihoodVar(only_gap=True, var = var_idx) )

            for i in range(n_rep):
                pred, targ, metric = imp.imp.preds_all_metrics(items = [items[i]], dls=dls, metrics=metrics_fn)
                pred, targ = pred[0], targ[0]
                pred = pred.mean.iloc[:, [var_idx]]
                targ = MeteoImpDf(targ.data.iloc[:, [var_idx]], targ.mask.iloc[:, [var_idx]], targ.control.iloc[:, [var_idx]])
                out = {
                    'var': var,
                    'loss': metric['loss'][0].item(),
                    'likelihood': metric['likelihood'][0].item(),
                    'gap_len': gap_len,
                    'idx_rep': i,
                } | imp.drop(index=["model", "imp"]).to_dict()
                for aggr in self.aggrs:
                    out = out | aggr(targ, pred, var)
                outs.append(out)
        return pd.DataFrame(outs)
    
    def compare(self, n_rep:int, gap_len:list[int], var:list[list[str]]):
        """Compare imputation performance for all combination of parameters"""
        arg_sets = list(product_dict(gap_len=tuplify(gap_len), var=tuplify(var)))
        out = []
        for arg_set in tqdm(arg_sets):
            out.append(self._compare_single(**arg_set, n_rep=n_rep))
        return prep_df(pd.concat(out))
        

In [ ]:
models = pd.DataFrame(columns = ["use_control", "gap_single_var"])

In [ ]:
base_path = here("analysis/results/trained_13feb")

In [ ]:
#| export
def l_model(x): return torch.load(base_path / x)

In [ ]:
models = [
    {'use_control': True,  'gap_single_var': True,  'model': l_model("1_gap_varying_12-336_v1.pickle")},
    {'use_control': True,  'gap_single_var': False, 'model': l_model("All_gap_all_30_v1.pickle")},
    {'use_control': False, 'gap_single_var': False, 'model': l_model("1_gap_varying_336_no_control_v1.pickle")},
    {'use_control': False, 'gap_single_var': True,  'model': l_model("1_gap_varying_336_no_control_v1.pickle")},
]
models = pd.DataFrame.from_records(models)

In [ ]:
kcomp = KalmanImpComparison(models, hai, hai_era, 100)

In [ ]:
#| export
from meteo_imp.kalman.fastai import _index_var

In [ ]:
models

,use_control,gap_single_var,model
0,True,True,"Kalman Filter\n N dim obs: 9,\n N dim state: 18,\n N dim contr: 14"
1,True,False,"Kalman Filter\n N dim obs: 9,\n N dim state: 18,\n N dim contr: 14"
2,False,False,"Kalman Filter\n N dim obs: 9,\n N dim state: 18,\n N dim contr: 14"
3,False,True,"Kalman Filter\n N dim obs: 9,\n N dim state: 18,\n N dim contr: 14"


In [ ]:
k_results = kcomp.compare(n_rep =3, gap_len = [6, 12, 24], var = list(hai.columns))

  0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
k_results

,var,loss,likelihood,gap_len,gap_idx,use_control,gap_single_var,rmse,rmse_stand
0,TA,-5.280393,2.671731,3.0,0,True,True,0.131880,0.016642
1,TA,-4.566644,2.487593,3.0,1,True,True,0.663002,0.083664
2,TA,-5.032340,2.686779,3.0,2,True,True,0.390265,0.049247
3,TA,-6.865580,4.571592,3.0,0,True,False,0.599223,0.075615
4,TA,-8.085504,4.509787,3.0,1,True,False,0.146722,0.018515
...,...,...,...,...,...,...,...,...,...
7,TS,34.731586,0.887256,12.0,1,False,False,2.722457,0.481115
8,TS,66.913388,0.769370,12.0,2,False,False,20.535604,3.629069
9,TS,7.724566,1.516849,12.0,0,False,True,1.946988,0.344073
10,TS,-0.383060,1.612699,12.0,1,False,True,0.809863,0.143120


## Plotting

In [ ]:
#| export
import altair as alt

In [ ]:
#| export
def facet_wrap(data: pd.DataFrame, # full dataset
               plot_fn,# function that makes the plot, takes 2 arguments: data and y_label
               col: str, # column to facet
               y_labels: list[str]|None =None, # custom labels y axis 
               n_cols=3,
               y_resolve='independent'
              ):
    col_vals = data[col].unique()
    plot_list = [alt.hconcat() for _ in range(0, len(col_vals), n_cols)]
    for i, col_v in enumerate(col_vals):
        plot = plot_fn(data[data[col]==col_v].copy(),
                       y_labels[i] if y_labels is not None else col_v
                      ).properties(title=str(col_v))
        plot_list[i // n_cols] |= plot
    return alt.vconcat(*plot_list).resolve_scale(xOffset='independent')#.resolve_scale(
        #y=y_resolve
    #)
    

In [ ]:
#| export
def facet_grid(data: pd.DataFrame, # full dataset
               plot_fn,# function that makes the plot, takes 2 arguments: data and y_label
               col: str, # column to facet,
               row: str,
               y_labels: list[str]|None = None, # custom labels y axis
              ):
    row_vals = data[row].unique()
    n_cols = len(data[col].unique())
    plots = []
    for row_val, y_label in zip_longest(row_vals, listify(y_labels)):
        plot = facet_wrap(data[data[row]==row_val].copy(), plot_fn, col, y_label, n_cols=n_cols).properties(title=row_val)
        plots.append(plot)
    return alt.vconcat(*plots)

In [ ]:
from itertools import product

In [ ]:
test_data = pd.DataFrame(list(product(['0','1'], ['a', 'b'])), columns = ['row', 'col'])
test_data['text'] = test_data.row + test_data.col

In [ ]:
def test_plot(data, *args): return alt.Chart(data).mark_text().encode(text='text')

In [ ]:
facet_wrap(test_data, test_plot, col = 'row')

alt.VConcatChart(...)

In [ ]:
facet_grid(test_data, test_plot, col = 'col', row='row')

alt.VConcatChart(...)

In [ ]:
#| export
method_scale = alt.Scale(domain=["KalmanFilter", "ERA", "MDS"], scheme='dark2')
meteo_scale = alt.Scale(domain = ['TA', 'SW_IN', 'LW_IN', 'VPD', 'WS', 'PA', 'P', 'SWC', 'TS'], scheme='dark2')

### The Plot 

In [ ]:
def remove_outliers(data, var, groupby):
    """Simple but maybe computationally inefficent to remove outliers from boxplot """

In [ ]:
def custom_box_plot(data, x, y, color, xoffset):
    """Boxplot for altair without outliers"""
    bar = alt.Chart(data).mark_bar().encode(
            x=x,
            y='q1:Q',
            y2='q2:Q',
            color=color,
            xoffset=xoffset
        ).transform_aggregate(
            q1=f'q1({y})',
            q3=f'q1({y})',
            groupby=[color, xoffset]
    )
    
    return bar

In [ ]:
# custom_box_plot(data_results, 'gap_len', 'rmse', 'method', 'method')

In [ ]:
#| export
def _the_plot(data, y_label='rmse', y = 'rmse'):
    # don't include ERA in xOffset scale if is always missing
    xoffset_domain = ["KalmanFilter", "ERA", "MDS"] # if not data.query("method == 'ERA'")[y].isna().all() else ["KalmanFilter", "MDS"]
    return alt.Chart(data).mark_boxplot(extent="min-max").encode(
        x = alt.X('gap_len:N', title='gap_len [h]', axis=alt.Axis(labelAngle=0)),
        y = alt.Y(y, title=y_label, axis=alt.Axis(grid=True)),
        color=alt.Color('method:N', scale=method_scale),
        xOffset=alt.XOffset('method', scale=alt.Scale(domain=xoffset_domain)),
        # column=alt.Column('var')
    ).properties(width=250, height=200)

In [ ]:
#| export
def the_plot(data):
     return facet_wrap(data, _the_plot, "var", y_labels = [f"rmse {var} [{units_big[var]}]" for var in data['var'].unique()]).configure_legend(orient="bottom")

In [ ]:
the_plot(data_results)

alt.VConcatChart(...)

In [ ]:
#| export
def the_plot_stand(data):
    # data = data.query("method == 'KalmanFilter'")
    return alt.Chart(data).mark_boxplot(extent="min-max").encode(
        x = alt.X('var', title='gap_len [h]', axis=alt.Axis(labelAngle=0)),
        y = alt.Y('rmse_stand', title="RMSE Standardized", axis=alt.Axis(grid=True)),
        color=alt.Color('method', scale=method_scale),
        xOffset=alt.XOffset('method:N', scale= alt.Scale(domain=method_scale.domain)),
    ).properties(width=500, height=300).configure_legend(orient="bottom")

In [ ]:
#| export
def the_plot_stand2(data):
     return _the_plot(data, y='rmse_stand', y_label="RMSE Standardized").facet(alt.Row('var',sort=list(var_type.categories)) , columns=3)

In [ ]:
dict(zip(scale_meteo.domain, scale_meteo.range))

{'TA': '#1B9E77',
 'SW_IN': '#D95F02',
 'LW_IN': '#7570B3',
 'VPD': '#E7298A',
 'WS': '#66A61E',
 'PA': '#E6AB02',
 'SWC': '#A6761D',
 'TS': '#666666'}

In [ ]:
the_plot_stand(data_results)

alt.Chart(...)

In [ ]:
the_plot_stand2(data_results)

alt.FacetChart(...)

In [ ]:
alt.Color('color')

Color({
  shorthand: 'color'
})

In [ ]:
x = alt.X('gap_len'); color = alt.Color('method'); xOffset = alt.XOffset('method'); y = alt.Y('rmse_stand')

In [ ]:
data_results.groupby([x.shorthand, color.shorthand, xOffset.shorthand]).agg({y.shorthand: ['median', lambda x: x.quantile(.25), lambda x: x.quantile(.75)]}).columns

MultiIndex([('rmse_stand',     'median'),
            ('rmse_stand', '<lambda_0>'),
            ('rmse_stand', '<lambda_1>')],
           )

In [ ]:
data_results

,method,var,gap_len,idx_rep,rmse,rmse_stand
0,KalmanFilter,TA,2.5,0,0.458599,0.057870
3,KalmanFilter,TA,2.5,1,0.156532,0.019753
6,KalmanFilter,TA,2.5,2,0.174399,0.022007
9,KalmanFilter,TA,2.5,3,0.361323,0.045595
12,KalmanFilter,TA,2.5,4,0.655402,0.082705
...,...,...,...,...,...,...
2,MDS,SW_IN,22.5,0,187.993906,0.921527
5,MDS,SW_IN,22.5,1,106.935300,0.524186
8,MDS,SW_IN,22.5,2,29.948688,0.146805
11,MDS,SW_IN,22.5,3,170.957608,0.838017


In [ ]:
data_results.groupby(['gap_len', 'method', 'var']).agg(
        median = pd.NamedAgg(y.shorthand, 'median'),
        q1 = pd.NamedAgg(y.shorthand,lambda x: x.quantile(.25)),
        q3 = pd.NamedAgg(y.shorthand,lambda x: x.quantile(.75))).reset_index()

,gap_len,method,var,median,q1,q3
0,2.5,KalmanFilter,TA,0.045595,0.022007,0.057870
1,2.5,KalmanFilter,SW_IN,0.128317,0.050479,0.241264
2,2.5,KalmanFilter,LW_IN,NaN,NaN,NaN
3,2.5,KalmanFilter,VPD,NaN,NaN,NaN
4,2.5,KalmanFilter,WS,NaN,NaN,NaN
...,...,...,...,...,...,...
103,22.5,MDS,WS,NaN,NaN,NaN
104,22.5,MDS,PA,NaN,NaN,NaN
105,22.5,MDS,P,NaN,NaN,NaN
106,22.5,MDS,SWC,NaN,NaN,NaN


In [ ]:
#| export
def custom_boxplot_nooutlier(data: pd.DataFrame,x:alt.X,y:alt.Y, color:alt.Color, xOffset:alt.XOffset):
    data = data.groupby(['var', 'method']).agg(
        median = pd.NamedAgg(y.shorthand, 'median'),
        q1 = pd.NamedAgg(y.shorthand,lambda x: x.quantile(.25)),
        q3 = pd.NamedAgg(y.shorthand,lambda x: x.quantile(.75))).reset_index().astype({'var': str})
    y_label= 'RMSE Standaridized'
    bar = alt.Chart(data).mark_bar(size=14).encode(alt.Y('q1', title=y_label), alt.Y2('q3', title=y_label), x, color, xOffset)
    tick = alt.Chart(data).mark_tick(color='white', size=14).encode(alt.Y('median'), x, xOffset)
    return bar + tick

In [ ]:
#| export
def the_plot_stand3(data):
    return custom_boxplot_nooutlier(data,
                                    alt.X('var:N', title='variable',axis=alt.Axis(labelAngle=0), scale=alt.Scale(domain=meteo_scale.domain)),
                                    alt.Y('rmse_stand'),
                                    alt.Color('method:N', scale=method_scale),
                                    alt.XOffset('method', )).properties(width=600)

In [ ]:
the_plot_stand3(data_results)

alt.LayerChart(...)

violin plots don't work

In [ ]:
def _the_plot_violin(data, y_label):
        return alt.Chart(data).mark_area(orient='horizontal').encode(
        x = alt.X('density:Q', title='gap_len [h]', axis=alt.Axis(labelAngle=0)),
        y = alt.Y('rmse', title=y_label, scale=alt.Scale(domainMin=-.2 *data['rmse'].mean())),
        color=alt.Color('method:N', scale=alt.Scale(domain=["KalmanFilter", "ERA", "MDS"], scheme='dark2')),
        xOffset=alt.XOffset('method', scale=alt.Scale(domain=["KalmanFilter", "ERA", "MDS"])),
        column=alt.Column('gap_len')
    ).transform_density(
    'rmse',
    as_=['rmse', 'density'],
    groupby=['method', 'gap_len']
).properties(width=250, height=200)

In [ ]:
facet_wrap(data_results, _the_plot_violin, "var", y_labels = [f"RMSE {var} [{units_big[var]}]" for var in data_results['var'].unique()])

alt.VConcatChart(...)

### Gap Length

In [ ]:
# 18,20,25,30,40,50,60,70
gap_len = ImpComparison(models_var, hai, hai_era, block_len=100).compare(gap_len = range(2,30,4), var=['TA', 'SW_IN', 'VPD'], n_rep=5)

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
#| export
def agg_gap_len(data):
    return pl.from_pandas(data).groupby(['var', 'gap_len']).agg([
        pl.col("rmse").median().alias("median"),
        pl.col("rmse").quantile(.25).alias("Q1"),
        pl.col("rmse").quantile(.75).alias("Q3"),
        pl.col("rmse").min().alias("min"),
        pl.col("rmse").max().alias("max"),
    ]).to_pandas() 

In [ ]:
gap_len_agg = agg_gap_len(gap_len)
# ).with_column(
#     (pl.col("Q3") - pl.col("Q1")).alias("IRQ")
# ).with_columns([
#     (pl.col("Q1") - 1.5 * pl.col("IRQ")).alias("min"),
#     (pl.col("Q3") + 1.5 * pl.col("IRQ")).alias("max"),
# ])

In [ ]:
#| export
def _get_era_rmse(df, control):
    """Average RMSE for ERA"""
    names = [col for col in control.columns if not col.endswith("_lag_1")]
    control = control.copy().filter(names)
    control = control.rename(columns=lambda x: x.replace("_ERA", "")).loc[df.index]
    rmse_df = np.sqrt((df[control.columns] - control).pow(2).mean(axis=0))
    rmse_df = rmse_df.to_frame().reset_index()
    rmse_df.columns = ["var", "era_rmse"] 
    return rmse_df

In [ ]:
rmse_df = _get_era_rmse(hai, hai_era)

In [ ]:
gap_len_agg = pd.merge(gap_len_agg, rmse_df, on='var')

In [ ]:
#| export
def _plot_gap_len(data, y_label):
    median = alt.Chart(data).mark_line().encode(
        x = alt.X('gap_len', title="gap_len [h]", axis=alt.Axis(labelAngle=0)),
        y = alt.Y('median', title=y_label),
        color=alt.Color('var', scale=meteo_scale)
    ).properties(height=150, width=200)
    Qs = alt.Chart(data).mark_errorband().encode(x = 'gap_len', y = alt.Y('Q1', title=y_label), y2= 'Q3', color='var')
    # min = alt.Chart(gap_len_agg).mark_point().encode(x = 'gap_len', y = 'min', color='var')
    # max = alt.Chart(gap_len_agg).mark_point().encode(x = 'gap_len', y = 'max', color='var')
    plot = (median + Qs)
    if not np.isnan(data.era_rmse.iloc[0]): plot += alt.Chart().mark_rule(strokeDash=[2,4]).encode(y=alt.datum(data.era_rmse.iloc[0]))

    return plot

In [ ]:
_plot_gap_len(gap_len_agg[gap_len_agg['var'] == 'TA'][gap_len_agg['method'] == 'ERA'], "label")

/tmp/ipykernel_101448/837074991.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  _plot_gap_len(gap_len_agg[gap_len_agg['var'] == 'TA'][gap_len_agg['method'] == 'ERA'], "label")


alt.LayerChart(...)

In [ ]:
facet_wrap(gap_len_agg[gap_len_agg.method == 'KalmanFilter'], _plot_gap_len, 'var')

alt.VConcatChart(...)

In [ ]:
#| export
def plot_gap_len(data, df, control):
    data = agg_gap_len(data.astype({'var': str, 'gap_len': str}))
    data = pd.merge(data, _get_era_rmse(df, control), on='var', how='left')
    # data = data[data.method == 'KalmanFilter']
    data = data.astype({'gap_len' : float, 'var': var_type}).sort_values(['var'])
    return facet_wrap(data, _plot_gap_len, 'var', y_labels = [f"RMSE {var} [{units_big[var]}]" for var in data['var'].unique()]).configure_legend(orient="bottom")

In [ ]:
plot_gap_len(gap_len, hai, hai_era)

ValueError: median encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.VConcatChart(...)

we actually don't care of different methods

In [ ]:
_plot_gap_len(gap_len_agg, y_label="").facet(column='method', row='var').resolve_scale(y='independent')

alt.FacetChart(...)

### Control

In [ ]:
#| export
from functools import partial
def _plot_compare(data, y_label='rmse', compare:str="", y = "rmse_stand", scale_domain=None):
    domain = ifnone(scale_domain, data[compare].unique())
    return alt.Chart(data).mark_boxplot(extent="min-max").encode(
        x = alt.X('gap_len:N', title='gap length [h]', axis=alt.Axis(labelAngle=0)),
        y = alt.Y(y, title=y_label, scale= alt.Scale(zero=False)),
        color=alt.Color(compare, scale=alt.Scale(domain = domain, scheme='accent')),
        xOffset=alt.XOffset(compare, scale=alt.Scale(domain = domain)),
        # column=alt.Column('var')
    ).properties(width=250, height=200)

In [ ]:
#| export
def plot_compare(data, compare, scale_domain=None):
     return facet_wrap(data, partial(_plot_compare, compare=compare, scale_domain=scale_domain),
                       "var", y_labels = [f"Standardized RMSE" for var in data['var'].unique()]
                      ).configure_legend(orient="bottom")

In [ ]:
#| export
def plot_control(data):
     return facet_wrap(data, partial(_plot_compare, compare='use_control'), "var", y_labels = [f"RMSE {var} [{units_big[var]}]" for var in data['var'].unique()])

In [ ]:
plot_control(k_results)

SchemaValidationError: Invalid specification

        altair.vegalite.v5.schema.core.Scale->0, validating 'type'

        True is not of type 'null'
        True is not of type 'string'
        True is not of type 'number'
        True is not of type 'boolean'
        True is not of type 'object'
        True is not of type 'object'
        

In [ ]:
#| export
def plot_control_loss(data):
     return facet_wrap(data, partial(_plot_compare, compare='use_control', y='loss'), "var", y_labels = [f"nll" for var in data['var'].unique()])

In [ ]:
plot_control_loss(k_results)

In [ ]:
#| export
def plot_control_lh(data):
     return facet_wrap(data, partial(_plot_compare, compare='use_control', y='likelihood'), "var", y_labels = [f"likelihood" for var in data['var'].unique()])

In [ ]:
plot_control_lh(k_results)

### Gap Multiple variables

In [ ]:
#| export
def plot_other_var(data):
     return facet_wrap(data, partial(_plot_compare, compare='gap_single_var', y='rmse'), "var", y_labels = [f"rmse {var} [{units_big[var]}]" for var in data['var'].unique()])

In [ ]:
plot_other_var(k_results)

### Single var vs Generic

In [ ]:
base_path = here("analysis/results/trained_14feb")

In [ ]:
#| export
def l_model(x): return torch.load(base_path / x)

In [ ]:
models_generic = [
    {'type': 'Generic',  'var': 'TA', 'model': l_model("1_gap_varying_12-336_v1.pickle")},
    {'type': 'Generic',  'var': 'SW_IN', 'model': l_model("1_gap_varying_12-336_v1.pickle")},
    {'type': 'Only one var', 'var': 'TA',    'model': l_model("TA_specialized_gap_12-336_v2.pickle.pickle")},
    {'type': 'Only one var', 'var': 'SW_IN', 'model': l_model("SW_IN_specialized_gap_12-336_v2.pickle.pickle")},
]
models_generic = pd.DataFrame.from_records(models_generic)

In [ ]:
@dataclass
class Col:
    col_name: str

In [ ]:
# dream_models_TA = [
#     Col('type'),  Col('model'),
#     'only TA',  l_model("trained_4_feb_All_gap_varying_12_v2.pickle"),
#     'generic',  l_model("trained_4_feb_TA_gap_12_v1.pickle"),
# ]

In [ ]:
# tribble ... not for now

In [ ]:
comp_generic = KalmanImpComparison(models_generic, hai, hai_era, 100)

In [ ]:
k_results_generic = comp_generic.compare(n_rep =3, gap_len = [6, 12, 24], var = ['TA', 'SW_IN'])

In [ ]:
k_results_generic

In [ ]:
#| export
def plot_generic(data):
     return facet_wrap(data, partial(_plot_compare, compare='type', y='rmse'), "var", y_labels = [f"rmse {var} [{units_big[var]}]" for var in data['var'].unique()])

In [ ]:
_plot_compare(k_results_generic, compare='type')

In [ ]:
plot_generic(k_results_generic)

### Short gaps vs Generic

In [ ]:
# models_short_gap = [
#     {'type': 'Short gap',  'var': 'TA', 'model': l_model("trained_4_feb_All_gap_varying_12_v2.pickle")},
#     {'type': 'Long gap',  'var': 'TA', 'model': l_model("trained_4_feb_TA_gap_12_v1.pickle")},
# ]
# models_short_gap = pd.DataFrame.from_records(models_short_gap)

### Timeseries

In [ ]:
comp_ts = ImpComparison(models = models_var, df = hai, control = hai_era, block_len = 100, rmse=False, time_series = True)

In [ ]:
res_ts = comp_ts.compare(gap_len = [5,10,20], var=["TA", 'SW_IN'], n_rep=5) 

In [ ]:
res_ts.columns

In [ ]:
test_ts = res_ts.query('method == "KalmanFilter" and var == "TA"')

In [ ]:
row_res = test_ts.iloc[0]

In [ ]:
pred_all = row_res.pred[0]
targ =  row_res.targ[0]
var = row_res['var']

In [ ]:
type(pred_all)

In [ ]:
targ.mask[var]

In [ ]:
list(row_res.index)

In [ ]:
np.argmin(row_res.targ[0].mask['TA'])

In [ ]:
row_res.targ[0].mask['TA'].index[47]

In [ ]:
#| export
def unnest_predictions(row_res: pd.Series, ctx_len:int=50):
    pred = row_res.pred[0]
    targ =  row_res.targ[0]
    var = row_res['var']
    if isinstance(pred, NormalsDf):
        mean = pred.mean[var]
        std = pred.std[var]
    else:
        mean = pred[var]
        std = np.nan
    
    
    measurement = targ.data[var]
    is_present = targ.mask[var]
    
    gap_len = (~is_present).sum()
    gap_start = np.argmin(is_present)
    ctx_start = is_present.index[gap_start - (ctx_len - gap_len) //2]
    ctx_end = is_present.index[gap_start + gap_len + (ctx_len - gap_len) //2]
    
    is_present = is_present[ctx_start:ctx_end]
    measurement = measurement[ctx_start:ctx_end]
    mean = mean[ctx_start:ctx_end]
    if type(std) != float: std = std[ctx_start:ctx_end] 
    
    mean[is_present] = np.nan # plot only predictions in gap
    
    other_cols = {name: row_res[name] for name in list(row_res.index) if name not in ['pred', 'targ']}
    
    out =  pd.DataFrame({'mean': mean, 'std': std, 'measurement': measurement, 'is_present': is_present} | other_cols)
    return out.reset_index()
    

In [ ]:
unnest_predictions(row_res)

In [ ]:
res_ts_plot = pd.concat([unnest_predictions(row) for _,row in res_ts.iterrows()])

In [ ]:
#| export
from meteo_imp.kalman.fastai import def_selection, plot_points, plot_line, plot_error

In [ ]:
plot_points(res_ts_plot.query('var == "TA" and method == "KalmanFilter" and idx_rep == 0 and gap_len==2.5'), y= "measurement")

In [ ]:
plot_missing_area(res_ts_plot.query('var == "TA" and method == "KalmanFilter" and idx_rep == 0 and gap_len==2.5'))

In [ ]:
plot_line(res_ts_plot.query('var == "TA" and idx_rep == 0 and gap_len==2.5'), y= "mean", color='method', scale=alt.Scale())

In [ ]:
plot_error(res_ts_plot.query('var == "TA" and idx_rep == 0 and gap_len==2.5').copy(), y= "mean", color='method', scale=alt.Scale())

In [ ]:
#| export
def _plot_timeseries(data_plot, y_label="", scale_color=method_scale, compare = 'method'):
    """data for one variable and one gap"""
    data_measure = data_plot[data_plot[compare] == data_plot[compare].unique()[0]]
    points = plot_points(data_measure, y= "measurement", y_label=y_label)
    miss_area = plot_missing_area(data_measure)
    err_band = plot_error(data_plot, y= "mean", color=compare, scale=scale_color, y_label=y_label)
    line = plot_line(data_plot, y= "mean", color=compare, scale=scale_color, y_label=y_label, props={'height': 200, 'width': 300})
    return (points + miss_area  + err_band + line)

In [ ]:
_plot_timeseries(res_ts_plot.query('var == "TA" and idx_rep == 0 and gap_len==2.5').copy(), "TA")

In [ ]:
_plot_timeseries(res_ts_plot.query('var == "SW_IN" and idx_rep == 0 and gap_len==10').copy(), "SW_IN")

In [ ]:
data_ts = pd.concat([unnest_predictions(row) for _,row in res_ts.iterrows()])

In [ ]:
#| export
def plot_timeseries(data, idx_rep:int|None=None, gap_len:int|None = None, max_idx:int = 3,
                    ctx_len={6.: 50, 12.: 50, 168.: 336+48},
                    scale_color=method_scale, compare='method'):
    
    if idx_rep == 'random': idx_rep = int(data['idx_rep'].sample())
    if gap_len is None and idx_rep is not None:
        data_plot = data.query(f'idx_rep == {idx_rep}').copy()
        facet_var = 'gap_len'
    elif gap_len is not None and idx_rep is None:
        data_plot = data.query(f'gap_len == {gap_len} and idx_rep < {max_idx}').copy()
        facet_var = 'idx_rep'
    else:
        raise ValueError(f"One and only one of idx_rep, gap_len should be None. got {idx_rep}, {gap_len}")
    data_plot = pd.concat([unnest_predictions(row, ctx_len[row.gap_len]) for _, row in data_plot.iterrows()])
    data_plot = data_plot.astype({'idx_rep': str, 'gap_len': str})
    data_plot = data_plot.assign(gap_len = data_plot.gap_len.map(lambda x: f"gap {x} h"))
    y_labels = [[f"rmse {var} [{units_big[var]}]" for _ in data_plot[facet_var].unique()] for var in data_plot['var'].unique()]
    return facet_grid(data_plot, partial(_plot_timeseries, scale_color=scale_color, compare=compare),
                      row="var", col=facet_var, y_labels=y_labels).configure_legend(orient="bottom")
        
    

In [ ]:
plot_timeseries(res_ts, idx_rep='random')

### Scatter plot

In [ ]:
#| export
def _plot_scatter(df, only_present=True, x = "value", y="mean", x_label="", y_label = "", color = 'method', scale=method_scale, props = {}):
    # df = df[df.is_present] if only_present else df
    # TODO remove onle_present
    return alt.Chart(df).mark_point().encode(
        x = alt.X(x, title=x_label),    
        y = alt.Y(y, title = y_label),
        color=alt.Color(color, scale= scale),
        shape = color
    ).properties(
        **props
    )

    

In [ ]:
data_ts.query('var == "TA" and gap_len==2.5').copy()

In [ ]:
_plot_scatter(data_ts.query('var == "TA" and gap_len==10 and idx_rep ==0').copy(), x="measurement", y= "mean", color='method', scale=alt.Scale())

## Table

### The Table

A table where in the rows there is 

In [ ]:
t = data_results.groupby(['method', 'var', 'gap_len']).agg({'rmse': ['mean', 'std']}).unstack(level=0)

t_idx = t.columns.droplevel()
t_idx.names = ['RMSE', None]

t2 = t.copy()
t2.columns = t_idx

t2 = t2.sort_index(axis=1, level=1).swaplevel(axis=1)
t2

NameError: name 'data_results' is not defined

In [ ]:
row = t2.iloc[0]

NameError: name 't2' is not defined

In [ ]:
row

NameError: name 'row' is not defined

In [ ]:
np.argmin(row.iloc[[0,2,4]]) * 2

NameError: name 'row' is not defined

In [ ]:
#| export
def highlight_min_method(row, props, cols): 
    # select even columns that are the mean
    return np.where(row == np.min(row.iloc[cols]), props, '')

def style_the_table(style, cols=[0,2,4]):
    return (style.apply(highlight_min_method, props="font-weight: bold", cols=cols, axis=1)
                .format_index(precision=0).format(precision=3, na_rep='-'))

In [ ]:
# #| export
# renames_table_latex = {name: f"\parbox{{2.1cm}}{{{val}}}" for name, val in 
#                  {'SW_IN': "Shortwave radiation incoming \\textbf{SW IN} [\si{W/m^2}]",
#                'LW_IN': 'Longwave radiation incoming \\textbf{LW IN} [$W/m^2$]',
#                'TA': "Air Temperature \\textbf{TA} [$°C$]",
#                'VPD': "Vapuour Pressure Deficit \\textbf{VPD} [$hPa$]",
#                'PA': "Air Pressure \\textbf{PA} [$hPa$]",
#                'P': "Precipitation \\textbf{P} [$mm$]",
#                'WS': "Wind Speed \\textbf{WS} [$m/s$]",
#           }.items()}

In [ ]:
#| export
renames_table_latex = {name: f"\\parbox{{2.1cm}}{{{val}}}" for name, val in 
                 {'SW_IN': "\\textbf{SW\\_IN} [\\si{W/m^2}]",
               'LW_IN': '\\textbf{LW\\_IN} [\\si{W/m^2}]',
               'TA': "\\textbf{TA} [\\si{°C}]",
               'VPD': "\\textbf{VPD} [\\si{hPa}]",
               'PA': "\\textbf{PA} [\\si{hPa}]",
               'P': "\\textbf{P} [\\si{mm}]",
               'WS': "\\textbf{WS} [\\si{m/s}]",
               'TS': "\\textbf{TS} [\\si{°C}]",
               'SWC': "\\textbf{SWC} [\\si{\%}]",
          }.items()}

In [ ]:
#| export
renames_table_latex_stand = {name: f"\\parbox{{2.1cm}}{{{val}}}" for name, val in 
                 {'SW_IN': "\\textbf{SW\\_IN}",
               'LW_IN': '\\textbf{LW\\_IN}',
               'TA': "\\textbf{TA}",
               'VPD': "\\textbf{VPD}",
               'PA': "\\textbf{PA}",
               'P': "\\textbf{P}",
               'WS': "\\textbf{WS}",
               'TS': "\\textbf{TS}",
               'SWC': "\\textbf{SWC}",
          }.items()}

In [ ]:
t2.style.pipe(style_the_table)

In [ ]:
t3 = t2.rename(index = renames_table_latex).style.apply(highlight_min_method, props="font-weight: bold", axis=1).format_index(precision=0).format(precision=3)

In [ ]:
t3

In [ ]:
print(t3.to_latex(convert_css=True, hrules=True, clines="skip-last;data", column_format="p{2.1cm}c|rr|rr|rr", caption="caption", label="table"))

In [ ]:
#| export
def the_table(data, y='rmse', y_name="RMSE"):
    data = data.groupby(['method', 'var', 'gap_len']).agg({y: ['mean', 'std']}).unstack(level=0)

    data_cols = data.columns.droplevel()
    data_cols.names = [y_name, None]
    data.columns = data_cols
    data.index.names = ["Variable", "Gap [$h$]"]
    return data.sort_index(axis=1, level=1).swaplevel(axis=1)   

In [ ]:
the_table(data_results)

In [ ]:
#| export
def the_table_latex(table, file, caption="", label="", stand=False):
    renames = renames_table_latex if not stand else renames_table_latex_stand
    styled = table.rename(index = renames).style.pipe(style_the_table).format(na_rep="-", precision=3)
    latex = styled.to_latex(convert_css=True, hrules=True, clines="skip-last;data",
                            column_format="p{2.1cm}c|rr|rr|rr", caption=caption, label=label, position_float="centering")
    with open(file, 'w') as f:
        f.write(latex)
    return file

In [ ]:
the_table_latex(the_table(data_results), "test_table.tex")

### Table compare

In [ ]:
#| export
err_type = CategoricalDtype(categories=["std", "mean", "diff."], ordered=True)
def table_compare(data, compare:str, y = 'rmse_stand', compare_ascending=True):
    data = data.groupby([compare, 'var', 'gap_len']).agg({y: ['mean', 'std']}).unstack(level=0).droplevel(level=0, axis=1)
    
    data["diff."] = (data.iloc[:, 0] - data.iloc[:, 1])
    
    data_cols = data.columns
    data_cols.names = ['RMSE Standardized', compare]
    # support custom sorting order
    data.columns = pd.MultiIndex.from_frame(data_cols.to_frame().astype({'RMSE Standardized': err_type}))
    data.index.names = ["Variable", "Gap [$h$]"]
    return data.sort_index(axis=1, level=1, ascending=False).swaplevel(axis=1) 

In [ ]:
table_compare(k_results, 'use_control')

use_control            False                True                    
RMSE Standardized       mean       std      mean       std     diff.
Variable Gap [$h$]                                                  
TA       3.0        0.071531  0.080346  0.042377  0.031936  0.029154
         6.0        0.254662  0.154330  0.079730  0.044252  0.174932
         12.0       0.518039  0.435547  0.127106  0.091420  0.390933
SW_IN    3.0        0.126355  0.103091  0.302880  0.222587  0.176525
         6.0        0.531083  0.468922  0.259730  0.144197  0.271353
         12.0       0.651510  0.359853  0.309855  0.252639  0.341655
LW_IN    3.0        0.117033  0.081575  0.218013  0.157215  0.100980
         6.0        0.260864  0.127380  0.210059  0.164418  0.050804
         12.0       0.648803  0.225924  0.531998  0.237676  0.116805
VPD      3.0        0.080883  0.040920  0.061606  0.016222  0.019277
         6.0        0.206404  0.118284  0.079571  0.051189  0.126833
         12.0       0.386051  0.262311  0.250002  0.192650  0.136050
WS       3.0        0.300485  0.158529  0.502712  0.560973  0.202226
         6.0        0.533166  0.333133  0.456949  0.250417  0.076217
         12.0       0.576789  0.276542  0.572316  0.360507  0.004473
PA       3.0        0.053446  0.045544  0.034332  0.014184  0.019114
         6.0        0.154374  0.070122  0.104465  0.080144  0.049909
         12.0       0.388000  0.335828  0.080165  0.059638  0.307834
P        3.0        0.057974  0.046056  0.471970  0.828965  0.413996
         6.0        0.453741  0.447474  0.252101  0.107018  0.201640
         12.0       0.452073  0.504036  0.436604  0.381016  0.015469
SWC      3.0        0.057809  0.039351  0.037269  0.029069  0.020541
         6.0        0.260036  0.172584  0.039388  0.030187  0.220648
         12.0       0.383962  0.195432  0.270892  0.302199  0.113070
TS       3.0        0.120985  0.071894  0.059529  0.015747  0.061456
         6.0        0.422910  0.397447  0.213845  0.156284  0.209065
         12.0       0.887094  1.348264  0.246851  0.171585  0.640243

In [ ]:
d

RMSE Standardized       mean       std      mean       std     diff.
use_control            False     False      True      True          
Variable Gap [$h$]                                                  
TA       3.0        0.071531  0.080346  0.042377  0.031936  0.029154
         6.0        0.254662  0.154330  0.079730  0.044252  0.174932
         12.0       0.518039  0.435547  0.127106  0.091420  0.390933
SW_IN    3.0        0.126355  0.103091  0.302880  0.222587  0.176525
         6.0        0.531083  0.468922  0.259730  0.144197  0.271353
         12.0       0.651510  0.359853  0.309855  0.252639  0.341655
LW_IN    3.0        0.117033  0.081575  0.218013  0.157215  0.100980
         6.0        0.260864  0.127380  0.210059  0.164418  0.050804
         12.0       0.648803  0.225924  0.531998  0.237676  0.116805
VPD      3.0        0.080883  0.040920  0.061606  0.016222  0.019277
         6.0        0.206404  0.118284  0.079571  0.051189  0.126833
         12.0       0.386051  0.262311  0.250002  0.192650  0.136050
WS       3.0        0.300485  0.158529  0.502712  0.560973  0.202226
         6.0        0.533166  0.333133  0.456949  0.250417  0.076217
         12.0       0.576789  0.276542  0.572316  0.360507  0.004473
PA       3.0        0.053446  0.045544  0.034332  0.014184  0.019114
         6.0        0.154374  0.070122  0.104465  0.080144  0.049909
         12.0       0.388000  0.335828  0.080165  0.059638  0.307834
P        3.0        0.057974  0.046056  0.471970  0.828965  0.413996
         6.0        0.453741  0.447474  0.252101  0.107018  0.201640
         12.0       0.452073  0.504036  0.436604  0.381016  0.015469
SWC      3.0        0.057809  0.039351  0.037269  0.029069  0.020541
         6.0        0.260036  0.172584  0.039388  0.030187  0.220648
         12.0       0.383962  0.195432  0.270892  0.302199  0.113070
TS       3.0        0.120985  0.071894  0.059529  0.015747  0.061456
         6.0        0.422910  0.397447  0.213845  0.156284  0.209065
         12.0       0.887094  1.348264  0.246851  0.171585  0.640243

In [ ]:
d.columns

MultiIndex([('mean', False),
            ( 'std', False),
            ('mean',  True),
            ( 'std',  True),
            (    '',    '')],
           names=['RMSE Standardized', 'use_control'])

In [ ]:
d.columns

MultiIndex([('rmse_stand', 'mean', False),
            ('rmse_stand', 'mean',  True),
            ('rmse_stand',  'std', False),
            ('rmse_stand',  'std',  True)],
           names=[None, None, 'use_control'])

In [ ]:
d.iloc[:, 0] - 

rmse_stand          
                    mean          
use_control        False     True 
var   gap_len                     
TA    3.0       0.071531  0.042377
      6.0       0.254662  0.079730
      12.0      0.518039  0.127106
SW_IN 3.0       0.126355  0.302880
      6.0       0.531083  0.259730
      12.0      0.651510  0.309855
LW_IN 3.0       0.117033  0.218013
      6.0       0.260864  0.210059
      12.0      0.648803  0.531998
VPD   3.0       0.080883  0.061606
      6.0       0.206404  0.079571
      12.0      0.386051  0.250002
WS    3.0       0.300485  0.502712
      6.0       0.533166  0.456949
      12.0      0.576789  0.572316
PA    3.0       0.053446  0.034332
      6.0       0.154374  0.104465
      12.0      0.388000  0.080165
P     3.0       0.057974  0.471970
      6.0       0.453741  0.252101
      12.0      0.452073  0.436604
SWC   3.0       0.057809  0.037269
      6.0       0.260036  0.039388
      12.0      0.383962  0.270892
TS    3.0       0.120985  0.059529
      6.0       0.422910  0.213845
      12.0      0.887094  0.246851

In [ ]:
#| export
def table_compare_latex(table, file, caption="", label=""):
    styled = table.rename(index = renames_table_latex_stand).style.pipe(partial(style_the_table, cols=[0,2]))
    latex = styled.to_latex(convert_css=True, hrules=True, clines="skip-last;data",
                            column_format="p{2.1cm}c|rr|rr|r", caption=caption, label=label, position_float="centering")
    with open(file, 'w') as f:
        f.write(latex)
    return file

### Table gap single

In [ ]:
#| export
def table_compare3(data, compare:str, y = 'rmse_stand', compare_ascending=True):
    data = data.groupby([compare, 'var', 'gap_len']).agg({y: ['mean', 'std']}).unstack(level=0).droplevel(level=0, axis=1)
    
    data_cols = data.columns
    data_cols.names = ['RMSE Standardized', compare]
    data.index.names = ["Variable", "Gap [$h$]"]
    return data.sort_index(axis=1, level=1, ascending=True).swaplevel(axis=1) 

In [ ]:
#| export
def table_compare3_latex(table, file, caption="", label=""):
    styled = table.rename(index = renames_table_latex_stand).style.pipe(partial(style_the_table, cols=[0,2,4]))
    latex = styled.to_latex(convert_css=True, hrules=True, clines="skip-last;data",
                            column_format="p{2.1cm}c|rr|rr|rr", caption=caption, label=label, position_float="centering")
    with open(file, 'w') as f:
        f.write(latex)
    return file

### Gap len table

In [ ]:
g = gap_len.groupby(['var', 'gap_len']).agg({'rmse': ['mean']})

In [ ]:
g

In [ ]:
(g.droplevel(level=0, axis=1)
 .reset_index()
 .melt(id_vars=['var', 'gap_len'], var_name='rmse')
 .pivot(index = ['var', 'rmse'], columns=['gap_len'])
 .droplevel(level=0, axis=1)
)

In [ ]:
g.columns 

In [ ]:
#| export
def table_gap_len(data, y = 'rmse'):
    t = (data
         .groupby(['var', 'gap_len']).agg({y: ['mean', 'std']})
        .droplevel(level=0, axis=1)
         .reset_index()
         .melt(id_vars=['var', 'gap_len'], var_name='rmse')
         .pivot(index = ['var', y], columns=['gap_len'])
         .droplevel(level=0, axis=1)
        ) 
    
    t.columns.names = ["Gap [$h$]"]
    t.index.names = ("Variable", "RMSE")
    
    return t.sort_index(axis=1, level=1)

In [ ]:
table_gap_len(gap_len)

NameError: name 'gap_len' is not defined

In [ ]:
#| export
def table_gap_len_latex(table, file, caption="", label=""):
    table.columns = [f"{col:.0f}" for col in list(table.columns)]
    styled = table.rename(index = renames_table_latex_stand).style.format(precision=3, na_rep='-')
    table_cols = 'c' * len(table.columns)
    latex = styled.to_latex(convert_css=True, hrules=True, clines="skip-last;data",
                            column_format="p{2.1cm}l|" + table_cols, caption=caption, label=label, position_float="centering")
    with open(file, 'w') as f:
        f.write(latex)
    return file

In [ ]:
[f"{col:.0f}" for col in list(table_gap_len(gap_len).columns)]

['1', '3', '5', '7', '9', '11', '13']

In [ ]:
table_gap_len_latex(table_gap_len(gap_len), "")

,,1,3,5,7,9,11,13
Variable,RMSE,,,,,,,
\parbox{2.1cm}{\textbf{TA} [\si{°C}]},mean,1.452,1.617,1.169,1.900,1.600,1.656,0.980
\parbox{2.1cm}{\textbf{SW\_IN} [\si{W/m^2}]},mean,32.173,47.047,61.438,33.085,48.179,40.442,48.532
\parbox{2.1cm}{\textbf{LW\_IN} [\si{W/m^2}]},mean,-,-,-,-,-,-,-
\parbox{2.1cm}{\textbf{VPD} [\si{hPa}]},mean,1.162,0.725,1.526,1.711,2.078,1.284,1.759
\parbox{2.1cm}{\textbf{WS} [\si{m/s}]},mean,-,-,-,-,-,-,-
\parbox{2.1cm}{\textbf{PA} [\si{hPa}]},mean,-,-,-,-,-,-,-
\parbox{2.1cm}{\textbf{P} [\si{mm}]},mean,-,-,-,-,-,-,-
\parbox{2.1cm}{\textbf{SWC} [\si{\%}]},mean,-,-,-,-,-,-,-
\parbox{2.1cm}{\textbf{TS} [\si{°C}]},mean,-,-,-,-,-,-,-
